# INDEXATION WEB : Python rev

## Import des modules

In [9]:
from collections import Counter
import numpy as np
import pandas as pd
import re
import nltk
nltk.download("stopwords")
from nltk.corpus import stopwords
##Stopwords
stops = set(stopwords.words("english"))
import os
import time
from collections import defaultdict
import pickle
import gzip

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
#with open("test.txt") as f:
#    test = f.read()

In [3]:
#import re
#list_test = re.split("\n|\t|\s|\.|\,|\;", test)
#list_test

In [4]:
#import pickle
#score = {"joueur1":5, "joueur2":10, "joueur3":15, "joueur4":20}
#with open("donnees", 'wb') as mon_fichier:
#    monPickler = pickle.Pickler(mon_fichier)
#    monPickler.dump(score)

#####"recuperer les objets sauvegarder
#with open("donnees", 'rb') as mon_fichier1:
#    monDepickler = pickle.Unpickler(mon_fichier1)
#    dico = monDepickler.load()

#print(dico)

In [5]:
#####"recuperer les objets sauvegarder
#with open("donnees", 'rb') as mon_fichier1:
#    monDepickler = pickle.Unpickler(mon_fichier1)
#    dico = monDepickler.load()

#print(dico)

<h1 style="color:blue; text-align:center">1. TEXT DATA LOADING AND CLEANING</h1>

<h3 style="color:blue">First, get the list of paths to all data files</h3>

In [8]:
##Function to retrieve all files' paths in a given directory
def getFiles(path):
    """The function will return a list of paths for all files
    contained in the <path> direcory"""
    
    res = []
    
    for root, directories, filenames in os.walk(path):
        for filename in filenames: 
            res.append("./" + os.path.join(root,filename))
    return res



liste_dirs = getFiles("data/")
liste_dirs[0], len(liste_dirs)

NameError: name 'os' is not defined

<h3 style="color:blue">Then, read the files and clean them</h3>

In [7]:
def load_Clean(list_paths):
    """For every file's path in the <list_paths>, the function read it,
    clean it from stopwords and apply le"""

In [8]:
l[0]

'data/KevinDrawbaugh/108321newsML.txt'

In [7]:
#LOading data in a list
liste_text = []
for path in liste_rep:
    with open(path, 'r') as f:
        text = f.read()
        liste_text.append(text)

NameError: name 'liste_rep' is not defined

In [8]:
len(liste_text)

2500

In [9]:
#liste_text[0]
article0 = liste_text[0].lower()
#liste_article0 = re.split("\n|\t|\s|\.|\,|\;|\:|", article0)
liste_article0 = re.findall("[a-z0-9]+", article0)
liste_article1 = [elt for elt in liste_article0 if elt not in stops]

start1 = time.time()
c = Counter(liste_article1)
sorted_List_counter = c.most_common(n=len(liste_article1))
end1 = time.time()
print(end1 - start1)

0.0001666545867919922


### Construction du vocabulaire global

In [3]:
def globalCounter(my_list):
    myCounter = Counter()
    for text in my_list:
        text = text.lower()
        list_word_withStops = re.findall("[a-z0-9]+", text)
        list_word = [elt for elt in list_word_withStops if elt not in stops]
        
        myCounter += Counter(list_word)
    
    return myCounter

In [4]:
myCounter = globalCounter(liste_text)

NameError: name 'liste_text' is not defined

In [12]:
myVocab = myCounter.most_common(10)
myVocab

[('said', 19856),
 ('year', 5386),
 ('percent', 5296),
 ('would', 5202),
 ('million', 4877),
 ('company', 3778),
 ('new', 3547),
 ('market', 3336),
 ('1', 3232),
 ('china', 3154)]

In [13]:
myCounter["canada"]

535

In [14]:
len(myCounter)

29024

# Création d'index inversé

In [15]:
##Récupérer La liste des mots de mon vocabulaire
list_vocab = list(myCounter.keys())
len(list_vocab)

29024

In [5]:
def cleanText(text):
    text = text.lower()
    text = re.findall("[a-z0-9]+", text)
    return ([elt for elt in text if elt not in stops])

In [6]:
clean_Docs = list( map( cleanText, liste_text ) )

NameError: name 'liste_text' is not defined

In [18]:
def create_index (data):
    index = defaultdict(list)
    for i, words in enumerate(data):
        for word in words:
            index[word].append(i)
    index_new = {}
    for word in index.keys():
        index_new[word] = dict( Counter( index[word] ) )
    
    return index_new

In [19]:
def create_index2(data):
    index = defaultdict(list)
    for i, words in enumerate(data):
        for word in words:
            index[word].append(i)
    index_new = {}
    for word in index.keys():
        index_new[word] = dict( Counter( index[word] ) )
    
    res = {}
    for word in index_new.keys():
        dico = index_new[word]
        for doc_num in dico.keys():
            doc = np.array(data[doc_num])
            pos = np.array((np.where(doc==word))).tolist()
            dico[doc_num] = {"occurencies":dico[doc_num], "positions":pos[0], "doc_size":len(data[doc_num])}
            
        res[word] = dico
            
    return res

In [85]:
##Optimize index creation time
def create_index3(data):
    index = defaultdict(list)
    res={}
    
    for i, words in enumerate(data):
        for word in words:
            index[word].append(i)
    index_new = {}
    for word in index.keys():
        index_new[word] = dict( Counter( index[word] ) )
        for doc_num in index_new[word].keys():
            pos = np.array((np.where(np.array(data[doc_num])==word))).tolist()
            index_new[word][doc_num] = {"occurencies":index_new[word][doc_num], "positions":pos[0], "doc_size":len(data[doc_num])}
        res[word] = index_new[word]
            
    return res

In [82]:
#testind = create_index(clean_Docs)
#testind["china"]
#dict(Counter([1,2,2,2,3,3,2,5,7]))

In [20]:
#newIndex["china"]

from time import time
start = time()
test_Index = create_index2(clean_Docs)
end = time()
print(end-start)

17.137162446975708


In [86]:
from time import time
start = time()
indexOpt = create_index3(clean_Docs)
end = time()
print(end-start)

16.753427982330322


# Serialize data with gzip

In [22]:
#store the object
with gzip.open("Index_Articles.pklz", "wb") as fout:
    pickle.dump(test_Index,fout)
    

#restore the object
with gzip.open("Index_Articles.pklz", "rb") as fin:
    newIndex = pickle.load(fin)

In [23]:
newIndex["china"]

{57: {'doc_size': 409, 'occurencies': 3, 'positions': [213, 259, 264]},
 85: {'doc_size': 246, 'occurencies': 1, 'positions': [216]},
 91: {'doc_size': 411, 'occurencies': 1, 'positions': [219]},
 111: {'doc_size': 265,
  'occurencies': 6,
  'positions': [14, 67, 82, 112, 170, 198]},
 169: {'doc_size': 332, 'occurencies': 1, 'positions': [326]},
 177: {'doc_size': 332, 'occurencies': 1, 'positions': [326]},
 210: {'doc_size': 279, 'occurencies': 1, 'positions': [272]},
 243: {'doc_size': 267, 'occurencies': 1, 'positions': [263]},
 264: {'doc_size': 268, 'occurencies': 1, 'positions': [200]},
 278: {'doc_size': 364, 'occurencies': 1, 'positions': [88]},
 300: {'doc_size': 366, 'occurencies': 2, 'positions': [140, 152]},
 303: {'doc_size': 387, 'occurencies': 1, 'positions': [181]},
 307: {'doc_size': 331, 'occurencies': 1, 'positions': [90]},
 309: {'doc_size': 339,
  'occurencies': 8,
  'positions': [8, 39, 50, 97, 105, 119, 138, 197]},
 323: {'doc_size': 452, 'occurencies': 3, 'posit

# Create Search function

In [24]:
def search_v1(query, index):
    query = query.lower()
    
    try :
        return list(index[query].keys())
    except :
        print("No result for specified key!!!")
        return None

In [26]:
query1 = search_v1("China", newIndex)
query1

[57,
 85,
 91,
 111,
 169,
 177,
 210,
 243,
 264,
 278,
 300,
 303,
 307,
 309,
 323,
 324,
 329,
 340,
 343,
 344,
 349,
 417,
 424,
 434,
 437,
 445,
 446,
 450,
 451,
 452,
 453,
 454,
 455,
 456,
 457,
 458,
 459,
 460,
 461,
 462,
 463,
 464,
 465,
 466,
 467,
 468,
 469,
 470,
 471,
 472,
 473,
 474,
 475,
 476,
 477,
 478,
 479,
 480,
 481,
 483,
 484,
 485,
 486,
 487,
 488,
 489,
 490,
 491,
 492,
 493,
 494,
 495,
 496,
 497,
 498,
 499,
 531,
 562,
 565,
 567,
 574,
 575,
 583,
 586,
 700,
 702,
 703,
 705,
 707,
 710,
 713,
 715,
 718,
 719,
 721,
 722,
 723,
 728,
 729,
 730,
 734,
 735,
 736,
 737,
 740,
 741,
 742,
 743,
 747,
 749,
 750,
 752,
 753,
 754,
 755,
 756,
 757,
 758,
 759,
 760,
 761,
 763,
 765,
 769,
 770,
 771,
 774,
 775,
 776,
 779,
 782,
 783,
 784,
 785,
 786,
 789,
 792,
 794,
 797,
 798,
 799,
 818,
 822,
 827,
 842,
 843,
 850,
 851,
 852,
 853,
 854,
 855,
 856,
 857,
 858,
 859,
 860,
 861,
 862,
 863,
 864,
 865,
 866,
 867,
 868,
 869,
 870,
 

In [27]:
def search_v2(query, index):
    item_list = re.findall("[a-z0-9]+", query)
    item_list = list(map(str.lower, item_list))
    res = []
    for item in item_list:
        tmp = search_v1(item, index)
        if tmp:
            res.extend(tmp)
    return list(set(res))

In [29]:
query2 = search_v2("china economy ", newIndex)
query2

[2050,
 2051,
 2052,
 2053,
 2054,
 2055,
 2056,
 2057,
 2058,
 2059,
 2060,
 2061,
 2062,
 2063,
 2064,
 2065,
 2066,
 2067,
 2068,
 2069,
 2070,
 2071,
 2072,
 2073,
 2075,
 2076,
 2077,
 2078,
 2079,
 2080,
 2081,
 2082,
 2083,
 2084,
 2085,
 2086,
 2087,
 2088,
 2089,
 2090,
 2091,
 2092,
 2093,
 2094,
 2095,
 2096,
 2097,
 2098,
 2099,
 57,
 67,
 2115,
 15,
 2125,
 2126,
 83,
 85,
 2138,
 91,
 2139,
 2140,
 96,
 2145,
 2146,
 2148,
 2153,
 111,
 119,
 2168,
 2171,
 2172,
 2175,
 2176,
 2186,
 2189,
 2192,
 2195,
 2197,
 2201,
 2202,
 2206,
 2207,
 2211,
 2212,
 2215,
 169,
 2219,
 2223,
 2224,
 177,
 180,
 2228,
 2232,
 37,
 2237,
 2240,
 2241,
 2249,
 210,
 2280,
 243,
 247,
 2301,
 2304,
 2305,
 2306,
 2307,
 2308,
 2309,
 2310,
 2311,
 264,
 2312,
 2313,
 2314,
 2315,
 2316,
 2317,
 2318,
 2319,
 2320,
 2321,
 2322,
 2323,
 2324,
 278,
 2325,
 2326,
 2327,
 2328,
 2329,
 2330,
 2331,
 2332,
 2333,
 2334,
 2335,
 2336,
 2337,
 2338,
 2339,
 2340,
 2341,
 2342,
 2343,
 2344,
 234

In [30]:
def getPosition(word, doc, index):
    try:
        return index[word][doc]["positions"]
    except:
        return None

In [31]:
print(getPosition("china", 57, newIndex))

[213, 259, 264]


In [33]:
def search_v3(query, index):
    item_list = cleanText(query)
    item_list2 = list(map(str.lower, item_list))
    res = set()
    final_Result = []
    for i in range(len(item_list2)):
        tmp = set(search_v1(item_list2[i], index))
        if i ==0:
            res = tmp.intersection(tmp)
        else :
            res = res.intersection(tmp)
        
    
    if len(res) > 0:
        list_pos = []
        isNear = []
        for doc_num in res:
            posMinusOne = []
            for item in item_list2:
                list_pos.append(getPosition(item, doc_num, index))
            for i, p in enumerate(list_pos) :
                posMinusOne.append( set([x-i for x in p]) )
                
            if (len(set.intersection(*posMinusOne)) > 0):
                final_Result.append(doc_num)
                
    return final_Result


In [34]:
query3 = search_v3("Other organisations have been targeted in the past. Last year, the Nation of Islam's Million Man March web site was vandalised. And hackers make 250,000 attempts annually to break into U.S.", newIndex)
query3

[1906]

In [35]:
liste_rep[1906]

'data/AaronPressman/2537newsML.txt'



## Autres façon de compter les fréquences des mots

def search_v3(query, index):
    item_list = re.findall("[A-Za-z0-9]+", query)
    item_list2 = list(map(str.lower, item_list))
    res = set()
    final_Result = []
    for i in range(len(item_list2)):
        tmp = set(search_v1(item_list2[i], index))
        if i ==0:
            res = tmp.intersection(tmp)
        else :
            res = res.intersection(tmp)
    
    if len(res) > 0:
        list_pos = []
        isNear = []
        for doc_num in res:
            for item in item_list2:
                list_pos.append(getPosition(item, doc_num, index))
            min_occurence = min ([len(i) for i in list_pos])
            firstDocPos = list_pos[0]
            list_other_Docs = list_pos[1:]
            for position in firstDocPos:
                for other_Pos in list_other_Docs:
                    for p in other_Pos:
                        isNear.append(abs(p-position))
            if (1 in isNear):
                final_Result.append(doc_num)
    return final_Result


<h1 style="color:red">Ordered Queries</h1>

### Orderd queries by absolute frequencies

In [51]:
def search_v1_sorted(query, index):
    query = query.lower()
    
    try :
        tmp = list(index[query].items())
        tmp = sorted( list(map(lambda x:(x[0], x[1]["occurencies"]), tmp)), key=lambda a:a[1], reverse=True)
        return list(map(lambda x:x[0], tmp))
    
    except :
        print("No result for specified key!!!")
        return None

In [52]:
search_v1_sorted("china", newIndex)

[1407,
 774,
 2342,
 2409,
 2449,
 871,
 1430,
 1447,
 1168,
 1762,
 799,
 1404,
 1751,
 1791,
 2061,
 2068,
 2445,
 729,
 775,
 1405,
 1406,
 1416,
 1428,
 1438,
 1443,
 1759,
 2428,
 454,
 863,
 890,
 1182,
 1421,
 1439,
 1444,
 1602,
 1620,
 1770,
 1795,
 1796,
 1799,
 2051,
 2422,
 462,
 464,
 1161,
 1169,
 1174,
 1194,
 1429,
 1449,
 1629,
 1766,
 1767,
 2060,
 2070,
 2311,
 2313,
 860,
 895,
 1170,
 1173,
 1186,
 1409,
 1418,
 1442,
 1600,
 1608,
 1610,
 1758,
 1772,
 1773,
 1784,
 1789,
 2052,
 2424,
 2431,
 452,
 474,
 740,
 771,
 797,
 850,
 866,
 872,
 886,
 888,
 889,
 1166,
 1175,
 1178,
 1189,
 1191,
 1199,
 1400,
 1417,
 1425,
 1426,
 1604,
 1630,
 1633,
 1753,
 1755,
 1756,
 1768,
 1798,
 2067,
 2079,
 2082,
 2317,
 2412,
 2415,
 2439,
 2441,
 2442,
 309,
 459,
 485,
 765,
 769,
 776,
 789,
 856,
 862,
 869,
 875,
 882,
 893,
 897,
 899,
 1155,
 1157,
 1180,
 1184,
 1408,
 1410,
 1423,
 1440,
 1613,
 1624,
 1754,
 1757,
 1765,
 1775,
 2085,
 2096,
 2332,
 2420,
 2429,
 4

In [47]:
liste_rep[1407], liste_rep[774], liste_rep[2342]

("data/LynneO'Donnell/275670newsML.txt",
 'data/SarahDavison/405565newsML.txt',
 'data/GrahamEarnshaw/277010newsML.txt')

### Ordered queries by weighted frequencies

In [53]:
def search_v1_weighted(query, index):
    query = query.lower()
    
    try :
        tmp = list(index[query].items())
        tmp = sorted( list(map(lambda x:(x[0], x[1]["occurencies"]/x[1]["doc_size"]), tmp)),
                     key=lambda a:a[1], reverse=True)
        return list(map(lambda x:x[0], tmp))
    
    except :
        print("No result for specified key!!!")
        return None

In [61]:
search_v1_weighted("china", newIndex)
#newIndex["china"][871]

[2409,
 2449,
 871,
 740,
 454,
 1168,
 729,
 1406,
 1447,
 863,
 775,
 1762,
 799,
 2068,
 1182,
 1407,
 2428,
 1791,
 1602,
 1620,
 774,
 459,
 1194,
 1429,
 2445,
 1751,
 2422,
 2051,
 1169,
 1421,
 1439,
 872,
 1174,
 1633,
 1404,
 1430,
 2317,
 1770,
 1796,
 1405,
 2060,
 1187,
 2342,
 1184,
 2061,
 1191,
 2313,
 2070,
 1428,
 1155,
 1630,
 1189,
 1438,
 1443,
 1795,
 2052,
 1759,
 1170,
 1180,
 888,
 1157,
 481,
 462,
 474,
 1648,
 1629,
 1175,
 485,
 1418,
 1608,
 2431,
 1444,
 1610,
 1177,
 1186,
 890,
 1425,
 2442,
 1417,
 1799,
 797,
 895,
 1166,
 2311,
 869,
 1161,
 1183,
 1767,
 1625,
 2085,
 452,
 1449,
 1156,
 2424,
 1604,
 488,
 1640,
 2096,
 1789,
 2079,
 2067,
 852,
 771,
 1415,
 860,
 1426,
 1173,
 886,
 1772,
 850,
 495,
 856,
 1768,
 1758,
 457,
 889,
 1600,
 776,
 466,
 1402,
 789,
 866,
 1612,
 2412,
 1792,
 1624,
 1613,
 1153,
 498,
 2420,
 1798,
 1773,
 2153,
 2348,
 893,
 464,
 896,
 1400,
 2415,
 1784,
 2321,
 1757,
 1753,
 1408,
 2332,
 2082,
 1766,
 741,
 85

In [56]:
liste_rep[2409], liste_rep[2449], liste_rep[871]

('data/JaneMacartney/242319newsML.txt',
 'data/JaneMacartney/242519newsML.txt',
 'data/MureDickie/126632newsML.txt')

### ordered queries weighted by tf_idf 

In [68]:
def search_v1_tf_idf(query, index):
    query = query.lower()
    
    try :
        tmp = list(index[query].items())
        N = len(tmp)
        D = len(index)
        tmp = sorted( list(map(lambda x:(x[0], 
                                         (x[1]["occurencies"]/x[1]["doc_size"])*np.log(D/(1+N)),
                                        ), tmp)),
                     key=lambda a:a[1], reverse=True)
        return list(map(lambda x:x[0], tmp))
    
    except :
        print("No result for specified key!!!")
        return None

In [73]:
search_v1_tf_idf("economy", newIndex) == search_v1_weighted("economy", newIndex)

True

<h1 style="color:red; text-align:center">Ordering for free Text queries</h1>

In [ ]:
def search_v2_sorted(query, index):
    #item_list = re.findall("[a-z0-9]+", query)
    item_list = list(map(str.lower, re.findall("[a-z0-9]+", query)))
    res = []
    for item in item_list:
        tmp = search_v1(item, index)
        if tmp:
            res.extend(tmp)
    return list(set(res))

In [114]:
#newIndex
#len(newIndex)
len(list_vocab)
#clean_Docs[0]

['pall',
 'corp',
 'memtec',
 'ltd',
 'may',
 'different',
 'reasons',
 'seem',
 'equally',
 'determined',
 'acquire',
 'gelman',
 'sciences',
 'inc',
 'analysts',
 'arbitrageurs',
 'reluctant',
 'say',
 'whether',
 'portends',
 'bidding',
 'war',
 'especially',
 'rally',
 'american',
 'depositary',
 'shares',
 'memtec',
 'thursday',
 'reflected',
 'doubt',
 'wall',
 'street',
 'memtec',
 'commitment',
 'still',
 'gelman',
 'would',
 'nicely',
 'complement',
 'operations',
 'either',
 'suitor',
 'likely',
 'hotly',
 'pursued',
 'filtration',
 'industry',
 'analysts',
 'said',
 'shares',
 'ann',
 'arbor',
 'mich',
 'based',
 'gelman',
 'closed',
 '3',
 '29',
 'american',
 'stock',
 'exchange',
 'news',
 'discussions',
 'pall',
 'possible',
 'acquisition',
 'gelman',
 'reached',
 'definitive',
 'agreement',
 'august',
 '30',
 'acquired',
 'australia',
 'memtec',
 'said',
 'thursday',
 'intends',
 'proceed',
 'agreement',
 'think',
 'memtec',
 'going',
 'back',
 'away',
 'said',
 'robert'

<h1> Bag of Words</h1>

In [115]:
from sklearn.feature_extraction import DictVectorizer
from collections import Counter, OrderedDict
Corpus = list_vocab
v = DictVectorizer()

v.fit([OrderedDict.fromkeys(Corpus, 1)])
X = v.transform(Counter(f) for f in (clean_Docs))

#print(type(X))
#print(X.A)


In [130]:
print(X.A)

[[ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 ..., 
 [ 1.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]]


In [129]:
np.where(X.A[0]==1)

(array([  100,   112,   140,   223,   279,   407,   543,   561,   601,
          604,   734,  1391,  1418,  1474,  1563,  1834,  1835,  1842,
         1843,  2108,  2152,  2415,  2637,  2699,  2880,  2907,  3276,
         3277,  3383,  3429,  3497,  4937,  5366,  5799,  5805,  6110,
         6111,  6113,  6392,  6443,  6445,  6479,  6486,  6501,  6881,
         7229,  7938,  8128,  8130,  8266,  8379,  8584,  9245,  9456,
         9812,  9851,  9904, 10080, 10109, 10148, 10213, 10727, 10828,
        11194, 11246, 11757, 11871, 11893, 12113, 12676, 12806, 13048,
        13461, 13525, 13675, 13966, 13967, 14004, 14262, 14590, 14601,
        15204, 15229, 15909, 15957, 16225, 16308, 16369, 16527, 16657,
        16724, 16879, 16994, 17358, 17362, 17428, 17571, 17843, 17864,
        17866, 17900, 18481, 18769, 19068, 19086, 19167, 19448, 19719,
        19975, 20014, 20226, 20297, 20408, 20772, 20773, 20783, 21082,
        21129, 21204, 21443, 21668, 21893, 22128, 22363, 22890, 22892,
      

In [133]:
import numpy as np
from scipy.sparse import csr_matrix

docs = [["hello", "world", "hello", "alka"], ["goodbye", "cruel", "world"]]
indptr = [0]
indices = []
data = []
vocabulary = {}
for d in docs:
    for term in d:
        index = vocabulary.setdefault(term, len(vocabulary))
        indices.append(index)
        data.append(1)
    indptr.append(len(indices))

#csr_matrix((data, indices, indptr), dtype=int).toarray()

In [140]:
vocabulary, data, indptr

({'alka': 2, 'cruel': 4, 'goodbye': 3, 'hello': 0, 'world': 1},
 [1, 1, 1, 1, 1, 1, 1],
 [0, 4, 7])